In [2]:
from matplotlib import patches, pyplot, pyplot as plt
import cv2
import xml.etree.ElementTree as ET 
import numpy as np
import sys
from getBoundingBoxes import *
from getBoxFeatures import *
from corner_detector import *
from anms import *
from estimateAllTranslation import *
from applyBoxTransform import applyBoxTransform
import warnings
warnings.filterwarnings("ignore")

cap = cv2.VideoCapture('vids/Easy.mp4')
ret,firstFrame = cap.read()
boxes = getBoundingBoxes('first.xml')
gray = cv2.cvtColor(firstFrame,cv2.COLOR_BGR2GRAY)
boxesData = []
pts = 6
for box in boxes:
    boximg = gray[box[1]:box[3],box[0]:box[2]]
    x,y = getBoxFeatures(boximg,box,pts)
    boxData = {
        'coords': box,
        'x': x,
        'y': y,
        'startX': x.copy(),
        'startY': y.copy(),
        'startCoords': box,
        'valid': np.ones(x.shape[0],dtype='bool')
    }
    boxesData.append(boxData)
currentFrame = firstFrame
idx = 0
while(cap.isOpened()):
    ret,nextFrame = cap.read()
    if ret == False:
        break
    # do work
    gray = cv2.cvtColor(currentFrame,cv2.COLOR_BGR2GRAY)
    plt.imshow(cv2.cvtColor(nextFrame,cv2.COLOR_BGR2RGB))
    fig = plt.gcf()

    for boxData in boxesData:
        print('----',np.sum(boxData['valid']))
        if np.sum(boxData['valid'])>=pts:
            boxData['x'],boxData['y'],boxData['valid'] = estimateAllTranslation(boxData['x'],boxData['y'],boxData['valid'],currentFrame,nextFrame)
        else:
            print('inside else')
            boxData['startCoords']=boxData['coords']
            box = boxData['coords'].astype(int)
            boximg = gray[box[1]:box[3],box[0]:box[2]]
            boxData['x'],boxData['y'],boxData['valid'] = refreshFeatures(boximg,box,boxData['x'], boxData['y'], boxData['valid'],pts)
            boxData['startX']=boxData['x']
            boxData['startY']=boxData['y'] 
            print('----after refresh',np.sum(boxData['valid']))
        startX = boxData['startX']
        startY = boxData['startY']
        ax1 = fig.add_subplot(111)
        ax1.scatter(boxData['x'][boxData['valid']],boxData['y'][boxData['valid']],c='r',s=1)
        # print(np.sum(boxData['valid']),end=" ")
        if np.sum(boxData['valid']) >= 4:
            boxprev = boxData['coords']
            boxData['coords'] = applyBoxTransform(startX,startY,boxData['x'],boxData['y'],boxData['startCoords'],boxData['valid'])
            coords = boxData['coords']
            # xmin,ymin,xmax,ymax
            if coords[0] > 0 and coords[1] > 0 and coords[2] < currentFrame.shape[1] and coords[3] < currentFrame.shape[0]:
                rect = patches.Rectangle((coords[0],coords[1]),coords[2]-coords[0],coords[3]-coords[1],linewidth=1,edgecolor='b',facecolor='none')
                ax1.add_patch(rect)
#             else:
#                 print('homo fucked up')
#                 coords = boxprev
#                 boxData['coords']=boxprev
#                 rect = patches.Rectangle((coords[0],coords[1]),coords[2]-coords[0],coords[3]-coords[1],linewidth=1,edgecolor='b',facecolor='none')
#                 ax1.add_patch(rect)
    plt.savefig("outputs2/"+str(idx).zfill(4)+".png")
    fig.clf()
    idx = idx+1
    print(idx)
    currentFrame = nextFrame


---- 6
1
---- 6
2
---- 6
3
---- 6
4
---- 6
5
---- 6
6
---- 6
7
---- 6
8
---- 6
9
---- 6
10
---- 6
11
---- 6
12
---- 6
13
---- 6
14
---- 6
15
---- 6
16
---- 6
17
---- 6
18
---- 6
19
---- 6
20
---- 6
21
---- 6
22
---- 6
23
---- 6
24
---- 6
25
---- 6
26
---- 6
27
---- 6
28
---- 6
29
---- 6
30
---- 6
31
---- 6
32
---- 6
33
---- 6
34
---- 6
35
---- 6
36
---- 6
37
---- 6
38
---- 6
39
---- 6
40
---- 6
41
---- 6
42
---- 6
43
---- 6
44
---- 6
45
---- 6
46
---- 6
47
---- 6
48
---- 6
49
---- 6
50
---- 6
51
---- 6
52
---- 6
53
---- 6
54
---- 6
55
---- 6
56
---- 6
57
---- 6
58
---- 6
59
---- 6
60
---- 6
61
---- 6
62
---- 6
63
---- 6
64
---- 6
65
---- 6
66
---- 6
67
---- 6
68
---- 6
69
---- 6
70
---- 6
71
---- 6
72
---- 6
73
---- 6
74
---- 6
75
---- 6
76
---- 6
77
---- 6
78
---- 6
79
---- 6
80
---- 6
81
---- 6
82
---- 6
83
---- 6
84
---- 6
85
---- 6
86
---- 6
87
---- 6
88
---- 6
89
---- 6
90
---- 6
91
---- 6
92
---- 6
93
---- 6
94
---- 6
95
---- 6
96
---- 6
97
---- 6
98
---- 6
99
---- 6
100
---- 6
1

error: OpenCV(4.1.0) ../modules/core/src/matrix.cpp:757: error: (-215:Assertion failed) dims <= 2 && step[0] > 0 in function 'locateROI'


In [1]:
from anms_mod import *
def refreshFeatures(gray,box,x_oldd, y_oldd, valid_oldd,pts):
#     plt.imshow(gray)
#     plt.show()
    x_oldd = x_oldd - box[0]
    y_oldd = y_oldd - box[1]
    res = corner_detector(gray)
#     plt.imshow(res)
#     plt.show()
    x, y, valid = anms_mod(x_oldd,y_oldd,valid_oldd,res,pts)
    return x+box[0],y+box[1],valid

In [ ]:
a = np.asarray([True,False,True,True])
b = np.asarray([[1.1,2.2,3,4],[2,1,2,1]])
c = b.reshape(-1,1)
c.reshape(-1).shape
